In [15]:
import json

In [79]:
OBP_AVG = 0.327
SLG_AVG = 0.365

FPCT_AVG = {
    '2': 0.992,
    '3': 0.993,
    '4': 0.977, 
    '5': 0.929,
    '6': 0.964,
    '7': 0.977,
    '8': 0.988,
    '9': 0.976
}

POSITION_WEIGHT = {
    '2': {"batting": 0.95, "fielding": 1.05},
    '3': {"batting": 1.21, "fielding": 0.83},
    '4': {"batting": 0.97, "fielding": 1.03},
    '5': {"batting": 0.96, "fielding": 1.04},
    '6': {"batting": 0.94, "fielding": 1.06},
    '7': {"batting": 1.02, "fielding": 0.98},
    '8': {"batting": 1.04, "fielding": 0.96},
    '9': {"batting": 1.17, "fielding": 0.85},
}

OPPONENT_WEIGHT = {
    "brothers": 1.15,
    "lions": 1.11,
    "monkeys": 0.96, 
    "guardians": 0.93,
    "dragons": 0.85
}

In [26]:
fileRoot = "../clean/clean_all_"
teamNameMap = {
    "中信兄弟": "brothers", 
    "味全龍": "dragons",
    "富邦悍將": "guardians", 
    "統一7-ELEVEn獅": "lions",
    "樂天桃猿": "monkeys"
}

In [81]:
class Player:
    def __init__(self, name, data):
        self.name = name
        self.data = data

        # A_{ij}
        canPlay = []
        for i in range(9):
            if (i + 1) in data["fielding"]["pos"]:
                canPlay.append(1)
            else:
                canPlay.append(0)
        self.canPlay = canPlay

        # F_{ij}
        F = {}
        for i in data["fielding"]["pos"]:
            F[str(i)] = float(data["fielding"]["FPCT"][str(i)]) / FPCT_AVG[str(i)]
            if F[str(i)] == 1:
                F[str(i)] = 0.98
            if F[str(i)] < 0.8:
                F[str(i)] = 0.8
        self.F = F

In [82]:
allData = {}
for team in teamNameMap.keys():
    with open(fileRoot + team + ".json", encoding="utf-8") as f:
        temp = []
        for line in f:
            player = json.loads(line)
            temp.append(Player(player["name"], player["data"]))
        allData[teamNameMap[team]] = temp

brothers = allData["brothers"]
dragons = allData["dragons"]
guardians = allData["guardians"]
lions = allData["lions"]
monkeys = allData["monkeys"]

* $A_{ij}$: whether player $i$ can play position $j$

In [73]:
print(brothers[0].name, brothers[0].canPlay)

王威晨 [0, 0, 0, 1, 1, 0, 0, 0, 0]


* $B_i$

In [52]:
def calcOPS(OBP, SLG):
    return OBP / OBP_AVG + SLG / SLG_AVG - 1

In [83]:
player = brothers[0]
aField = "天母"
aMonth = "Apr"
aPitcher = "陳鴻文"
aOppo = "富邦悍將"

OPSseason = float(player.data["batting"]["season"]["OPS+"])
if OPSseason > 160:
    OPSseason = 100
if OPSseason < 0:
    OPSseason = 0
OPSmonth = calcOPS(
    float(player.data["batting"]["month"][aMonth]["OBP"]), 
    float(player.data["batting"]["month"][aMonth]["TB"]) / float(player.data["batting"]["month"][aMonth]["AB"])
)
OPSfield = calcOPS(
    float(player.data["batting"]["field"][aField]["OBP"]), 
    float(player.data["batting"]["field"][aField]["TB"]) / float(player.data["batting"]["field"][aField]["AB"])
)
OBPseason = float(player.data["batting"]["season"]["OBP"])
if OBPseason == 0:
    OBPseason = 1
OBPp = float(player.data["batting"]["vsP"]["data"][aOppo][aPitcher]) if player.data["batting"]["vsP"]["data"][aOppo][aPitcher] != '0.000' else OBPseason

B = (OPSseason + OPSmonth + OPSfield + OBPp * 100 / OBPseason) / 400
B

0.5030619352570793

* $F_{ij}$

In [84]:
player.F

{'4': 0.8, '5': 1.0563061476856834}

* $V_{ij}$

In [85]:
for i in player.data["fielding"]["pos"]:
    print(POSITION_WEIGHT[str(i)]["batting"] * B + POSITION_WEIGHT[str(i)]["fielding"] * player.F[str(i)])

1.311970077199367
1.5814978514399067
